In [14]:
import pandas as pd
import numpy as np
import yfinance as yf
import matplotlib.pyplot as plt

start_date = "2022-10-01"
end_date = "2024-10-01"

apple_data = yf.download("AAPL", start_date, end_date)
tesla_data = yf.download("TSLA", start_date, end_date)
pfizer_data = yf.download("PFE", start_date, end_date)
ford_data = yf.download("F", start_date, end_date)
exxon_data = yf.download("XOM", start_date, end_date)
spy_data = yf.download("SPY", start_date, end_date)
mcd_data = yf.download("MCD", start_date, end_date, progress = True)


apple_prices = apple_data['Close']
tesla_prices = tesla_data['Close']
pfizer_prices = pfizer_data['Close']
ford_prices = ford_data['Close']
exxon_prices = exxon_data['Close']
spy_prices = spy_data['Close']
mcd_prices = mcd_data['Adj Close']

sp500_data = yf.download('^GSPC', start_date, end_date)
sp500_close = sp500_data['Close']


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [15]:
# Calculate the 20-day moving average
moving_average_apple = apple_prices.rolling(window=20).mean()
moving_average_tesla= tesla_prices.rolling(window=20).mean()
moving_average_pfizer= pfizer_prices.rolling(window=20).mean()
moving_average_ford = ford_prices.rolling(window=20).mean()
moving_average_exxon= exxon_prices.rolling(window=20).mean()
moving_average_spy= spy_prices.rolling(window=20).mean()
moving_average_mcd= mcd_prices.rolling(window=20).mean()

In [16]:
#code for mean revision

def mean_reversion(prices, moving_average):
    if prices[-1] > moving_average[-1]:
        return "sell"
    elif prices[-1] < moving_average[-1]:
        return "buy"
    else:
        return "hold"

In [17]:


# function for backtesting
def backtest(prices, moving_average, initial_capital=10000, window_size=20):
    cash = initial_capital
    shares = 0
    portfolio_values = []

    for i in range(window_size, len(prices)):
        signal = mean_reversion(prices[:i+1], moving_average[:i+1])
        current_price = prices[i]

        if signal == "buy" and cash > current_price:
            # Buy as much stock as possible with the cash
            shares_to_buy = cash // current_price
            cash -= shares_to_buy * current_price
            shares += shares_to_buy
        elif signal == "sell" and shares > 0:
            # Sell all
            cash += shares * current_price
            shares = 0

        # calculate total portfolio value
        portfolio_value = cash + shares * current_price
        portfolio_values.append(portfolio_value)

    # put results in a DataFrame
    backtest_results = pd.DataFrame({
        'Date': prices.index[window_size:],
        'Portfolio Value': portfolio_values
    })
    backtest_results.set_index('Date', inplace=True)

    return backtest_results



In [18]:
apple_backtest = backtest(apple_prices,moving_average_apple )
tesla_backtest = backtest(tesla_prices, moving_average_apple)
pfizer_backtest = backtest(pfizer_prices, moving_average_pfizer)
ford_backtest = backtest(ford_prices, moving_average_ford)
exxon_backtest = backtest(exxon_prices, moving_average_exxon)
spy_backtest = backtest(spy_prices, moving_average_spy)
mcd_backtest = backtest(mcd_prices, moving_average_mcd)

KeyError: -1

In [ ]:

def create_plot(stock, ticker):
    plt.figure(figsize=(14,7))
    plt.plot(stock['Portfolio Value'], label='Portfolio Value', color='blue', linewidth=2)

    plt.title(f'Mean Reversion Strategy Backtest with {ticker.upper()} Stock', fontsize=16, fontweight='bold')
    plt.xlabel('Date', fontsize=14)
    plt.ylabel('Portfolio Value ($)', fontsize=14)


    plt.grid(visible=True, linestyle='--', alpha=0.6)
    plt.legend(fontsize=12, loc='upper left')
    plt.tight_layout()
    plt.show()

create_plot(apple_backtest, "AAPL")
create_plot(tesla_backtest, "TSLA")
create_plot(pfizer_backtest, "PFE")
create_plot(ford_backtest, "F")
create_plot(exxon_backtest, "XOM")
create_plot(spy_backtest, "SPY")
create_plot(mcd_backtest, "MCD")

In [ ]:
def plot_strategy_vs_control(backtest_results, sp500_data, stock_prices, ticker, initial_capital=10000):
    """
    Overlays the strategy portfolio value with S&P 500 performance and 'Buy and Hold' raw values.

    Parameters:
    - backtest_results: DataFrame containing the portfolio value over time.
    - sp500_data: Series of S&P 500 closing prices over time.
    - stock_prices: Series of stock prices for the stock being analyzed.
    - ticker: Stock ticker (string).
    - initial_capital: Starting amount of money for 'Buy and Hold'.
    """

    common_index = backtest_results.index.intersection(sp500_data.index).intersection(stock_prices.index)
    backtest_results = backtest_results.loc[common_index]
    #sp500_data = sp500_data.loc[common_index]
    stock_prices = stock_prices.loc[common_index]

    #  Buy and Hold Portfolio Value
    buy_and_hold_shares = initial_capital / stock_prices.iloc[0]
    buy_and_hold_value = stock_prices * buy_and_hold_shares


    sp500_normalized = sp500_data * initial_capital / sp500_data.iloc[0]

    # Plot all values
    plt.figure(figsize=(14, 7))
    plt.plot(backtest_results.index, backtest_results['Portfolio Value'], label='Mean Revision', color='blue', linewidth=2)
    #plt.plot(sp500_normalized.index, sp500_normalized, label='S&P 500 Index', color='orange', linestyle='--', linewidth=2)
    plt.plot(stock_prices.index, buy_and_hold_value, label='Buy and Hold ', color='green', linestyle='-.', linewidth=2)

    plt.title(f'Strategy for {ticker.upper()} vs. Control Strategy (Buy and Hold)', fontsize=16, fontweight='bold')
    plt.xlabel('Date', fontsize=14)
    plt.ylabel('Portfolio Value ($)', fontsize=14)
    plt.legend(fontsize=12)
    plt.grid(visible=True, linestyle='--', alpha=0.6)
    plt.tight_layout()
    plt.show()


In [ ]:

plot_strategy_vs_control(apple_backtest, sp500_close, apple_prices, "AAPL")
plot_strategy_vs_control(tesla_backtest, sp500_close, tesla_prices, "TSLA")
plot_strategy_vs_control(pfizer_backtest, sp500_close, pfizer_prices, "PFE")
plot_strategy_vs_control(ford_backtest, sp500_close, ford_prices, "F")
plot_strategy_vs_control(exxon_backtest, sp500_close, exxon_prices, "XOM")
plot_strategy_vs_control(mcd_backtest, sp500_close, mcd_prices, "MCD")


In [ ]:


stocks = {
    "AAPL": {
        "backtest": apple_backtest,
        "prices": apple_prices
    },
    "TSLA": {
        "backtest": tesla_backtest,
        "prices": tesla_prices
    },

        "PFE": {
        "backtest": pfizer_backtest,
        "prices": pfizer_prices
    },
            "PFE": {
        "backtest": pfizer_backtest,
        "prices": pfizer_prices
    },
                "XOM": {
        "backtest": exxon_backtest,
        "prices": exxon_prices
    },
                    "MCD": {
        "backtest": mcd_backtest,
        "prices": mcd_prices
    },
}

control_returns = []
mean_reversion_returns = []

# Calculate and store the returns for each stock
for stock, data in stocks.items():
    backtest = data["backtest"]
    mean_reversion_return = (
        (backtest['Portfolio Value'].iloc[-1] - backtest['Portfolio Value'].iloc[0])
        / backtest['Portfolio Value'].iloc[0]
    ) * 100

    prices = data["prices"]
    control_return = (
        (prices.iloc[-1] - prices.iloc[0]) / prices.iloc[0]
    ) * 100

    mean_reversion_returns.append(mean_reversion_return)
    control_returns.append(control_return)

# Print out the returns in a table format
print(f"{'Stock':<6} {'Control Return (%)':<20} {'Mean Reversion Return (%)':<25}")
print("-" * 55)
for stock, control, mean_reversion in zip(stocks.keys(), control_returns, mean_reversion_returns):
    print(f"{stock:<6} {control:<20.2f} {mean_reversion:<25.2f}")

# Plot the bar graph
x = np.arange(len(stocks))
width = 0.35

plt.figure(figsize=(10, 6))
plt.bar(x - width/2, control_returns, width, label='Control Strategy (Buy and Hold)', color='blue')
plt.bar(x + width/2, mean_reversion_returns, width, label='Mean Reversion Strategy', color='orange')

plt.xlabel('Stocks')
plt.ylabel('Returns (%)')
plt.title('Comparison of Returns: Control vs Mean Reversion Strategy')
plt.xticks(x, stocks.keys())
plt.legend()
plt.grid(axis='y', linestyle='--', alpha=0.7)

plt.tight_layout()
plt.show()